# Calculating Damkohler number for the simulations

## Calculate characteristic reaction time scales assuming first order rates
## Import residence time data


In [1]:
import os

#Import third party libraries
import pandas as pd
import numpy as np

#Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Import data science library
import data_reader.data_processing as proc
import analyses.transient as sta

In [3]:
#Load standard values required to navigate through the datasets
data_dir = "E:/Zenodo_spatial_heterogeneity/"
Regimes = ["Slow", "Medium", "Fast"]
vels = [0.00038,0.0038,0.038]
gw = 1

scdict = proc.masterscenarios() #master dictionary of all spatially heterogeneous scenarios that were run
ratenames = proc.masterrates("saturated")

#Domains
Trial = list(t for t,values in scdict.items())
#Reactive species of concerns
gvarnames = ["DOC","DO","Ammonium","Nitrate","Nitrogen","TOC"]

### Import residence times/breakthrough times from tracer dataset

In [4]:
chem_path_data = os.path.join(data_dir, "Results", "massflux_comparison_steadystate_BG.csv")
chemdata = pd.read_csv(chem_path_data)
print(chemdata.columns)
chemdata['Regime'] = chemdata['Regime'].replace({'Equal':'Medium'})
print(chemdata.Regime.unique())

Index(['Trial', 'Variance', 'Anisotropy', 'Regime', 'Chem', 'massflux_in',
       'massflux_out', 'delmassflux', 'reldelmassflux', 'normmassflux', 'Time',
       'fraction', 'spatial_normmassflux_base', 'spatial_reldelmassflux_base',
       'normmassflux_spatial_fraction', 'reldelmassflux_spatial_fraction'],
      dtype='object')
['Slow' 'Medium' 'Fast']


In [5]:
chemdatah = chemdata[chemdata.Trial=="H"]
print(chemdatah[["Trial","Chem","normmassflux_spatial_fraction"]])

    Trial      Chem  normmassflux_spatial_fraction
0       H       DOC                            1.0
1       H        DO                            1.0
2       H   Nitrate                            1.0
3       H  Ammonium                            1.0
4       H  Nitrogen                            1.0
5       H       TOC                            1.0
294     H       DOC                            1.0
295     H        DO                            1.0
296     H   Nitrate                            1.0
297     H  Ammonium                            1.0
298     H  Nitrogen                            1.0
299     H       TOC                            1.0
588     H       DOC                            1.0
589     H        DO                            1.0
590     H   Nitrate                            1.0
591     H  Ammonium                            1.0
592     H  Nitrogen                            1.0
593     H       TOC                            1.0


### Calculate pseudo first order reaction rate constant
- Load the chemican concentration profile at steady state
- Identify distance traveled for 63% normalized removal
- Ratio of this normalized removal and corresponding travel time is the reaction rate constant

In [6]:
row = []
for r in Regimes:
    print(r)
    if r=="Medium":
        reg="Equal"
    else:
        reg=r
    for t in Trial:
        filename = reg+"AR_0_NS-A"+t+"_df.npy"
        data = np.load(os.path.join(data_dir,"Data",filename))
        conctime, TotalFlow, Headinlettime = sta.conc_time (data,0,50,0,30, 51, gvarnames,"Saturated")
        for g in gvarnames:
            idx = gvarnames.index(g)
            inletconc = conctime[-1, 0, idx]
            outletconc = conctime[-1,-1,idx]
            normconc = outletconc/inletconc
            row.append([r,t,g,inletconc, outletconc, normconc])

Slow
Medium
Fast


In [7]:
data = pd.DataFrame.from_records(row, columns = ["Regime", "Trial", "Chem", "Conc_in", "Conc_out", "Normconc"])
for Reg in Regimes:
    for c in gvarnames:
        base = data.loc[(data.Regime == Reg) & (data.Chem == c) & (data.Trial == 'H')]['Normconc'].values[0]
        data.loc[(data.Regime == Reg) & (data.Chem == c), 'base'] = base
data['rel_normconc'] = data.Normconc/data.base

In [8]:
timedata = pd.merge(data, chemdata, on=['Regime','Trial','Chem'])
timedata["k"] = -1*np.log(timedata.Normconc)/timedata.Time
print(timedata[["Regime", "Chem", "k"]].groupby(["Regime", "Chem"]).mean())

                        k
Regime Chem              
Fast   Ammonium  0.068590
       DO        1.255664
       DOC       0.206500
       Nitrate   0.031839
       Nitrogen  0.026442
       TOC       0.143979
Medium Ammonium  0.012170
       DO        0.293545
       DOC       0.045650
       Nitrate   0.050710
       Nitrogen  0.036687
       TOC       0.029972
Slow   Ammonium  0.001206
       DO        0.029889
       DOC       0.004954
       Nitrate   0.007385
       Nitrogen  0.004477
       TOC       0.001988


In [9]:
datah = timedata[timedata.Trial=="H"]
print(datah[["Regime", "Trial","Chem", "k","reldelmassflux_spatial_fraction","Normconc","rel_normconc"]])

     Regime Trial      Chem         k  reldelmassflux_spatial_fraction  \
0      Slow     H       DOC  0.004378                              1.0   
1      Slow     H        DO  0.026612                              1.0   
2      Slow     H  Ammonium  0.001076                              1.0   
3      Slow     H   Nitrate  0.006698                              1.0   
4      Slow     H  Nitrogen  0.004116                              1.0   
5      Slow     H       TOC  0.001926                              1.0   
294  Medium     H       DOC  0.034932                              1.0   
295  Medium     H        DO  0.216597                              1.0   
296  Medium     H  Ammonium  0.009028                              1.0   
297  Medium     H   Nitrate  0.044971                              1.0   
298  Medium     H  Nitrogen  0.031774                              1.0   
299  Medium     H       TOC  0.023279                              1.0   
588    Fast     H       DOC  0.154048 

In [10]:
thresh = 0.05
steps = [500 * 0.005, 2 * 0.005, 2 * 0.0005]
nc = timedata[timedata["Normconc"] < thresh].index
for n in nc:
    r = timedata.iloc[n]["Regime"]
    r = reg
    if r=="Medium":
        reg="Equal"
    t = timedata.iloc[n]["Trial"]
    g = timedata.iloc[n]["Chem"]
    inletconc = timedata.iloc[n]["Conc_in"]
    filename = reg+"AR_0_NS-A"+t+"_df.npy"
    concdata = np.load(os.path.join(data_dir, "Data",filename))
    conctime, TotalFlow, Headinlettime = sta.conc_time (concdata,0,50,0,30, 51, [g],"Saturated")        
    tracer_dir = "X:/Saturated_flow/Steady_state/Tracer_studies/" + Reg + "AR/NS-A"+t+"/"
    tracerfile = "NS-A"+t+"_df.npy"
    tracerdata = np.load(os.path.join(tracer_dir,tracerfile))
    tracertime, TracerFlow, TracerHeadinlettime = sta.conc_time (tracerdata,0,50,0,30, 51, ["Tracer_study"],"Saturated")            
    idx5 = np.where(conctime[-1, :, 0]<thresh*inletconc)[0]
    if idx5.size != 0:
        point = idx5[0]
        loss = conctime[-1, point, 0]/inletconc
        timidx = np.where(np.round(tracertime[:, point, 0], 3) > 10)
        tim = steps[Regimes.index(r)] * timidx[0][0]
        k = -1*np.log(loss)/tim
        timedata.iloc[n, timedata.columns.get_loc("k")] = k
selected_k = "k"
timedata["tau"] = -np.log(0.37)/timedata[selected_k].astype(float)

AttributeError: 'DataFrame' object has no attribute 'tau63'

In [11]:
timedata["Da"] = timedata.Time/timedata.tau
timedata.to_csv(os.path.join(data_dir, "Results", "Da_BG.csv"))